In [15]:
import sys, importlib
# script 경로 window용
sys.path.append(r"C:\Users\bong2\OneDrive\Python_Codes\00_python_scripts") 
    
from glob import glob
import pandas as pd
import time
import shutil
import conn_db, helper
from selenium import webdriver
from selenium.webdriver.support.select import Select

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [11]:
webdriver_path = r"C:\Users\bong2\OneDrive\Python_Codes\98_drivers\chromedriver.exe"
driver = webdriver.Chrome(executable_path = webdriver_path) 

In [12]:
# 1. 관세청 홈페이지 들어가기
# driver = webdriver.Chrome()

driver.get('https://unipass.customs.go.kr/ets/index.do')
time.sleep(10)

# 2. 실제 통계 목록 있는 화면으로 들어가기
# 신성질별 수출입실적 메뉴 클릭.
time.sleep(10)
selector = '#maincont > div > article > ul > li:nth-child(1) > div > ul.leftArea.listype01 > li:nth-child(5) > a'
driver.find_element_by_css_selector(selector).click()
time.sleep(10)

In [13]:
start_date = '2021.01'
end_date = 0

In [17]:
# 조회기간 '월'로 변경
selector = '#TRS0104011Q_priodKind > option:nth-child(2)'
driver.find_element_by_css_selector(selector).click()

# 시작날짜 설정
driver.find_element_by_css_selector('#TRS0104011Q_priodFr').click()  # 시작날짜 선택 드룹다운 열기
Select(driver.find_element_by_css_selector('#TRS0104011Q_priodFr')).select_by_visible_text(start_date)  # 시작날짜 설정

In [28]:
# 종료날짜 설정. 조회기간 'to' 클릭해서 dropdown 열기
driver.find_element_by_css_selector('#TRS0104011Q_priodTo').click()
if end_date == 0:
    # 항상 처음 있는 월 선택
    Select(driver.find_element_by_css_selector('#TRS0104011Q_priodTo')).select_by_index(0)
else: # 종료날짜 설정
    Select(driver.find_element_by_css_selector('TRS0104011Q_priodTo')).select_by_visible_text(end_date)

driver.find_element_by_css_selector('#TRS0104011Q_detailTmpr').click()  # 세부성질 클릭

In [40]:
# 수출, 수입 선택후 다운로드
trade_types = {'expTpcd':'수출',
               'impTpcd':'수입'}
path = conn_db.get_path('신성질별_수출입_raw')

for trade_type in trade_types.keys(): 
    # 수출, 수입 선택
    driver.find_element_by_css_selector(f'#TRS0104011Q_{trade_type}').click()
    
    # 조회클릭
    selector = '#TRS0104011Q_fmSearch > div > footer > button'
    driver.find_element_by_css_selector(selector).submit()
    time.sleep(30)
    
    start = len(glob(folder+'*.xls'))
    try: # 다운로드 버튼 클릭
        driver.find_element_by_css_selector('#TRS0104011Q_downExcel_double').send_keys('\n')
    except: # 다운로드 버튼 재클릭
        time.sleep(5)
        driver.find_element_by_css_selector('#TRS0104011Q_downExcel_double').send_keys('\n')        
        
    while len(glob(folder+'*.xls'))==start:
        time.sleep(3)
        
        
    old_file = glob(folder+'*.xls')[0]
    new_file = path + f'신성질별 {trade_types[trade_type]}실적_{start_date[:4]}.xls'

    shutil.move(src=old_file, dst=new_file)